##Wczytanie bibliotek

In [12]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical


%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
cd "/content/drive/My Drive/Colab Notebooks/Computer vision - znaki drogowe/computer-vision_road-signs"

/content/drive/My Drive/Colab Notebooks/Computer vision - znaki drogowe/computer-vision_road-signs


##Wczytywanie danych

In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

##Formatowanie danych

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

##Budowanie modelu

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
   model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
   
   logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
   tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
   
   model.fit(
       X_train, y_train,
       batch_size=params_fit.get('batch_size', 128),
       epochs=params_fit.get('epochs', 5),
       verbose=params_fit.get('verbose', 1),
       validation_data=params_fit.get('validation_data', (X_train, y_train)),
       callbacks= [tensorboard_callback]
       )

   return model


def predict (model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)


In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding = 'same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024,  activation='relu'),
  Dropout(0.3),

  Dense(1024,  activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation = 'softmax'),
])


In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.5538 - accuracy: 0.3045 - val_loss: 0.9724 - val_accuracy: 0.6850
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.7863 - accuracy: 0.7490 - val_loss: 0.2157 - val_accuracy: 0.9423
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.3697 - accuracy: 0.8850 - val_loss: 0.0863 - val_accuracy: 0.9743
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2472 - accuracy: 0.9245 - val_loss: 0.0447 - val_accuracy: 0.9879
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.1801 - accuracy: 0.9463 - val_loss: 0.0358 - val_accuracy: 0.9912


0.9687074829931973

##Hyperopt

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding = 'same'),
  MaxPool2D(),
  Dropout(params['droput_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['droput_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding = 'same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['droput_cnn_block_three']),

  Flatten(),

  Dense(1024,  activation='relu'),
  Dropout(params['droput_dense_block_one']),

  Dense(1024,  activation='relu'),
  Dropout(params['droput_dense_block_two']),

  Dense(num_classes, activation = 'softmax'),
])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
   
  model.fit(
      X_train, y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [35]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'droput_cnn_block_one': hp.uniform('droput_cnn_block_one', 0.3, 0.5),
    'droput_cnn_block_two': hp.uniform('droput_cnn_block_two', 0.3, 0.5),
    'droput_cnn_block_three': hp.uniform('droput_cnn_block_three', 0.3, 0.5),

    'droput_dense_block_one': hp.uniform('droput_dense_block_one', 0.3, 0.7),
    'droput_dense_block_two': hp.uniform('droput_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest, 
    30,
    Trials()
)

{'batch_size': 140.0, 'droput_cnn_block_one': 0.41674960704888, 'droput_cnn_block_three': 0.47818212802446597, 'droput_cnn_block_two': 0.3199335343439382, 'droput_dense_block_one': 0.3343326063993267, 'droput_dense_block_two': 0.5831691760722897}
accuracy=0.9342403411865234
{'batch_size': 150.0, 'droput_cnn_block_one': 0.34151977019628765, 'droput_cnn_block_three': 0.482741769679388, 'droput_cnn_block_two': 0.32805834556683394, 'droput_dense_block_one': 0.497170281915756, 'droput_dense_block_two': 0.30270476476665764}
accuracy=0.956916093826294
{'batch_size': 150.0, 'droput_cnn_block_one': 0.3158725278002929, 'droput_cnn_block_three': 0.3159438113463153, 'droput_cnn_block_two': 0.44026319136581327, 'droput_dense_block_one': 0.47098426685488715, 'droput_dense_block_two': 0.5335072079798513}
accuracy=0.9498866200447083
{'batch_size': 140.0, 'droput_cnn_block_one': 0.388079114911033, 'droput_cnn_block_three': 0.3140049615495071, 'droput_cnn_block_two': 0.427812188512134, 'droput_dense_blo